In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
#####################
#Question 1-1
#####################
# Load the housing data
path_to_file = 'C:\\Users\\angel\\Downloads\\housing.csv'
data = pd.read_csv(path_to_file)

# Binary feature names
binary_cols = ['prefarea', 'airconditioning', 'hotwaterheating', 'basement', 'guestroom', 'mainroad']

# Encode binary features
def binary_encoder(column):
    return column.map({'no': 0, 'yes': 1})

# Apply binary encoding
data[binary_cols] = data[binary_cols].apply(binary_encoder)
data.drop(columns='furnishingstatus', inplace=True)

# Extract target variable
y = data.pop('price').values.reshape(-1, 1)

# Normalize features and target
feature_scaler = StandardScaler()
target_scaler = StandardScaler()

X_normalized = torch.tensor(feature_scaler.fit_transform(data), dtype=torch.float32)
y_normalized = torch.tensor(target_scaler.fit_transform(y), dtype=torch.float32)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_normalized, y_normalized, test_size = 0.2, train_size=0.8, random_state=42)


In [3]:
# Define neural network model
class SingleNN(nn.Module):
    def __init__(self, input_size):
        super(SingleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define Triple layer neural network model
class TripleNN(nn.Module):
    def __init__(self, input_size):
        super(TripleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 16)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

# Initialize models, loss function and optimizers
model_single = SingleNN(X_train.shape[1])
model_triple = TripleNN(X_train.shape[1])
criterion = nn.MSELoss()
optimizer_single = optim.Adam(model_single.parameters(), lr=0.001)
optimizer_triple = optim.Adam(model_triple.parameters(), lr=0.001)


In [ ]:
# Training function
def train_model(epochs, optimizer, model, criterion, train_data, val_data, train_labels, val_labels, model_name):
    train_losses = []
    val_losses = []
    for epoch in range(1, epochs + 1):
        # Training step
        model.train()
        train_pred = model(train_data)
        train_loss = criterion(train_pred, train_labels)
        
        # Validation step
        model.eval()
        with torch.no_grad():
            val_pred = model(val_data)
            val_loss = criterion(val_pred, val_labels)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        # Logging
        train_losses.append(train_loss.item())
        val_losses.append(val_loss.item())
        
        if epoch % 500 == 0:
            print(f'{model_name} - Epoch {epoch}, Training Loss: {train_loss.item()}, Validation Loss: {val_loss.item()}')

    return train_losses, val_losses

# Train the single layer model
train_losses_single, val_losses_single = train_model(
    epochs=5000,
    optimizer=optimizer_single,
    model=model_single,
    criterion=criterion,
    train_data=X_train,
    val_data=X_val,
    train_labels=y_train,
    val_labels=y_val,
    model_name='Single Layer'
)

# Train the three layer model
train_losses_triple, val_losses_triple = train_model(
    epochs=5000,
    optimizer=optimizer_triple,
    model=model_triple,
    criterion=criterion,
    train_data=X_train,
    val_data=X_val,
    train_labels=y_train,
    val_labels=y_val,
    model_name='Triple Layer'
)


Single Layer - Epoch 500, Training Loss: 0.18575306236743927, Validation Loss: 0.5125681161880493
Single Layer - Epoch 1000, Training Loss: 0.137409508228302, Validation Loss: 0.5602630972862244
Single Layer - Epoch 1500, Training Loss: 0.1036447212100029, Validation Loss: 0.6083882451057434
Single Layer - Epoch 2000, Training Loss: 0.07941083610057831, Validation Loss: 0.6650856137275696
Single Layer - Epoch 2500, Training Loss: 0.06767471879720688, Validation Loss: 0.7370940446853638
Single Layer - Epoch 3000, Training Loss: 0.05986711010336876, Validation Loss: 0.8024779558181763


In [ ]:
# Plotting the training and validation loss curves
plt.figure(figsize=(12, 6))
plt.plot(train_losses_single, label='Single Model Training Loss')
plt.plot(val_losses_single, label='Single Model Validation Loss')
plt.plot(train_losses_triple, label='Triple Layer Model Training Loss')
plt.plot(val_losses_triple, label='Triple Layer Model Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()


In [ ]:
#####################
#Question 2-2 #Triple Hidden LAYER NN 
#####################
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
# Data transformations for CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# DataLoader
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Neural network with three hidden layers
class ExtendedFCNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes):
        super(ExtendedFCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, num_classes)

    def forward(self, x):
        x = x.view(-1, input_size)  # Flatten the image
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

# Hyperparameters
input_size = 32 * 32 * 3  # CIFAR-10 images are 32x32x3
hidden_size1 = 512
hidden_size2 = 256
hidden_size3 = 128
num_classes = 10
num_epochs = 300
learning_rate = 0.001

# Initialize the network, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ExtendedFCNN(input_size, hidden_size1, hidden_size2, hidden_size3, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the network
start_time = time.time()
print(f'Starting training for {num_epochs} epochs...')

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epoch + 1) % 50 == 0:  # Print every 10 epochs
        avg_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_loss:.4f}')

training_time = time.time() - start_time
print(f'Training time: {training_time:.2f} seconds')

# Evaluating the network
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the 10000 test images: {accuracy:.2f}%')


In [ ]:
#####################
#Question 2-1; SINGLE HIDDEN LAYER NN
#####################
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Data transformations for CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# DataLoader
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Neural network with one hidden layer
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(-1, input_size)  # Flatten the image
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Parameters
input_size = 32 * 32 * 3  # CIFAR-10 images are 32x32x3
hidden_size = 512
num_classes = 10
num_epochs = 300
learning_rate = 0.001

# Initialize the network, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FCNN(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the network
start_time = time.time()
for epoch in range(1, num_epochs + 1):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the results every 50 epochs
    if epoch % 50 == 0:
        # Calculate training loss
        avg_train_loss = running_loss / len(train_loader)
        
        # Calculate evaluation accuracy
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total

        # Print results
        print(f'Epoch [{epoch}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Evaluation Accuracy: {accuracy:.2f}%')

training_time = time.time() - start_time
print(f'Training time: {training_time:.2f} seconds')